### Test

In [ ]:
import json
import librosa
import pydub
import torch
import wave

import numpy as np

from IPython.display import Audio
from os import path
from PIL import Image as PImage, ImageDraw as PImageDraw

from transformers import VitsModel, AutoTokenizer, pipeline

OBJECTS_DB_FILE_PATH = "./metadata/objects.json"
SR = 11025

MODEL_PT = "facebook/mms-tts-por"

model_pt = VitsModel.from_pretrained(MODEL_PT).to("cuda")
tokenizer_pt = AutoTokenizer.from_pretrained(MODEL_PT)

pipe_pt = pipeline(model=MODEL_PT, device="cuda")

In [ ]:
out = pipe_pt("um dois tres testando")
display(Audio(out["audio"], rate=out["sampling_rate"]))

In [ ]:
img_data = {}
with open(OBJECTS_DB_FILE_PATH, "r", encoding="utf8") as f:
  img_data = json.load(f)["images"]

for k,v in list(img_data.items())[:10]:
  inputs = tokenizer_pt(v["caption"]["pt"], return_tensors="pt").to("cuda")

  with torch.no_grad():
    output_pt = model_pt(**inputs).waveform

  output_pt = output_pt.cpu().data.numpy().squeeze()
  output_pt = librosa.resample(output_pt, orig_sr=model_pt.config.sampling_rate, target_sr=SR)
  #display(Audio(output_pt, rate=SR))

  mp3 = pydub.AudioSegment(np.int16(output_pt * 2 ** 15).tobytes(), frame_rate=SR, sample_width=2, channels=1)
  mp3.export(f"{k}.mp3", format="mp3", bitrate="32k")